In [1]:
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
import pandas as pd

In [2]:
p1 = '../data/WN18RR/'
p2 = p1 + 'semantic/'
R = 11

In [5]:
r = [0]*R
train = pd.read_csv(p1+'train2id.txt', sep=' ', header=0, names=[1,2,3])
for i in range(R):
    tmp1 = np.array(train[train[3]==i])
    print(len(np.unique(tmp1[:,0])),end='\t')
    print(len(np.unique(tmp1[:,1])))

34033	9500
16102	16109
2466	404
707	787
3095	7340
2972	309
1978	3990
25	594
114	873
978	980
77	76


In [6]:
sem = np.loadtxt(p2+'ent2semantic.txt', delimiter=' ')
eww = np.loadtxt(p2+'entWithWords.txt', dtype=int)

In [7]:
# sem = (semantic - np.min(semantic)) / (np.max(semantic) - np.min(semantic))
eid2sem = dict(zip(eww, sem))

In [8]:
# 有关键字的实体
ent_set = set(eww)

In [9]:
train = pd.read_csv(p1+'train2id.txt', sep=' ', header=0, names=[1,2,3])
test = pd.read_csv(p1+'test2id.txt', sep=' ', header=0, names=[1,2,3])

In [10]:
def func1(x, k):
    maxsc = 0
    lb = []
    for i in range(5):
        tmp = KMeans(n_clusters=k).fit_predict(x)
        sc = metrics.calinski_harabasz_score(x, tmp)
        if maxsc < sc:
            maxsc = sc
            lb = tmp
    return lb, maxsc

def write_np(arr, p, sep=' '):
    with open(p, 'w', encoding='utf-8') as f:
        f.write(f'{len(arr)}\n')
        for i in arr:
            line = ''
            for j in i:
                line += f'{int(j)}{sep}' 
            line = line.strip() + '\n'
            f.write(line)
def normalize(arr):
    arr = (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
    return arr

***
对根据文本向量对头尾实体进行聚类，并记录结果

In [11]:
res = np.zeros([1,5], dtype=int)
tmp = train.copy(deep=True)
tmp[4] = 0
tmp[5] = 0
k=10 # 聚类簇的个数

box_num = np.zeros([R,2], dtype=int)
start_idx = np.zeros([R,2], dtype=int)
box_tot = np.zeros([2], dtype=int)
nh = 0
nt = 0

for rid in range(R):
    tmp1 = np.array(tmp[tmp[3] == rid])

    km = KMeans(n_clusters=k, random_state=10)
    
    h = np.unique(tmp1[:,0])
    t = np.unique(tmp1[:,1])
    
    if rid > 0:
        start_idx[rid,0] = start_idx[rid-1,0] + hb
        start_idx[rid,1] = start_idx[rid-1,1] + tb
    
    hb = 0
    tb = 0
    hpdic = {}
    tpdic = {}
    
    flag = 1
    tmp_eid = []
    tmp_sem = np.zeros([128])
    for i in h:
        if i in ent_set:    # 判断是否具有文本信息
            tmp_eid.append(i)
            tmp_sem = np.vstack((tmp_sem, eid2sem.get(i)))
        elif flag:
            # 如果存在实体没有文本信息，那就为这类实体创建一个box
            flag = 0
            hb += 1
    tmp_sem = np.delete(tmp_sem, 0, axis=0) # 删除第一行的全零
    if len(tmp_eid) >= k:   # 如果带有文本信息的实体数量>=聚类簇数，则进行聚类
        tmp_sem = normalize(tmp_sem)
        try:
            label, sc = func1(tmp_sem, k)   # label代表实体对应的boxid
        except Exception:
            label = km.fit_predict(tmp_sem)
        if not flag:
            label += 1
        hpdic = dict(zip(tmp_eid, label))
        hb += k
    elif len(tmp_eid) > 0:
        if not flag:
            hpdic = dict(zip(tmp_eid, np.ones(len(tmp_eid), dtype=int)))
        hb += 1
    
    flag = 1
    tmp_eid = []
    tmp_sem = np.zeros([128])
    for i in t:
        if i in ent_set:
            tmp_eid.append(i)
            tmp_sem = np.vstack((tmp_sem, eid2sem.get(i)))
            # tmp_sem.append(eid2sem.get(i))
        elif flag:
            flag = 0
            tb += 1
    tmp_sem = np.delete(tmp_sem, 0, axis=0)
    if len(tmp_eid) >= k:
        tmp_sem = normalize(tmp_sem)
        try:
            label, sc = func1(tmp_sem, k)
        except Exception:
            label = km.fit_predict(tmp_sem)
        if not flag:
            label += 1
        tpdic = dict(zip(tmp_eid, label))
        tb += k
    elif len(tmp_eid) > 0:
        if not flag:
            tpdic = dict(zip(tmp_eid, np.ones(len(tmp_eid), dtype=int)))
        tb += 1
    
    box_num[rid, 0] = hb
    box_num[rid, 1] = tb
    box_tot[0] += hb
    box_tot[1] += tb
    
    for i in range(len(tmp1)):
        h = tmp1[i][0]
        t = tmp1[i][1]

        if hpdic.get(h): # 即使取到0也没关系，因为初值就是0，可以不修改
            tmp1[i][3] = hpdic.get(h)
        if tpdic.get(t):
            tmp1[i][4] = tpdic.get(t)
    res = np.concatenate([res, tmp1])
res = np.delete(res, 0, axis=0)

In [12]:
res.shape

(86835, 5)

In [13]:
version = f'k{k}/'

In [14]:
write_np(res, f'{p2}{version}train2id.txt')
# np.savetxt(f'{p2}{version}train2id.txt', res,delimiter=' ', fmt='%d')
np.savetxt(f'{p2}{version}box_num.txt', box_num, delimiter=' ', fmt='%d')
np.savetxt(f'{p2}{version}box_tot.txt', box_tot, delimiter=' ', fmt='%d')
np.savetxt(f'{p2}{version}start_idx.txt', start_idx, delimiter=' ', fmt='%d')

In [15]:
train = pd.read_csv(f'{p2}k{k}/train2id.txt', sep=' ', header=None, names=[1,2,3,4,5])
tmp = test.copy(deep=True)
tmp[4] = -1
tmp[5] = -1
res = np.zeros([1,5], dtype=int)
for i in range(346):
    # 根据rid选择triple
    tmp1 = train[train[3] == i]
    tmp2 = np.array(tmp[tmp[3] == i])
#     print(f'{tmp1.shape}\t{tmp2.shape}', end='\t')
    
    # 获得头尾实体与具体box的对应关系
    hdict = dict(zip(tmp1[1].values, tmp1[4].values))
    tdict = dict(zip(tmp1[2].values, tmp1[5].values))
    
    for j in range(len(tmp2)):
        h,t,r,hb,tb = tmp2[j]
        if h in hdict.keys():
            tmp2[j][3] = hdict.get(h)
        
        if t in tdict.keys():
            tmp2[j][4] = tdict.get(t)
    
    res = np.concatenate([res, tmp2])
res = np.delete(res, 0, axis=0)
print(res.shape)

(3134, 5)


In [16]:
write_np(res, f'{p2}k{k}/test2id.txt')